In [1]:
def Gumbel_for_one(series_of_values, name):
    """Calculation of Gumbel distribution for any characteristics.
          
        series_of_values: pd.series, import values
        
        name: str, name of characteristic
    """
    
    table1 = pd.read_csv('table_of_n_y.csv', delimiter = ';')
    table2 = pd.read_csv('table_of_n_sigma.csv', delimiter = ';')

    from scipy import interpolate
    inter_y = interpolate.interp1d(table1['n'], table1['y_mean'])
    inter_sigma = interpolate.interp1d(table2['n'], table2['sigma'])
    
    # 
    data = series_of_values
    
    MSE = data.std()
    mean = data.mean()
    y = inter_y(len(data))
    sigma = inter_sigma(len(data))
    q = mean - y * (MSE / sigma)

    p_values = [0.01, 0.1, 1, 5, 10, 20, 30, 50, 63, 70, 80, 90, 95, 99, 99.9]
    
    p_df = pd.DataFrame(data=p_values, columns=['P_%'])

    from numpy import log as ln
    p_df['Yp'] = -ln(-ln((100 - p_df['P_%']) / 100))
    p_df[name] = q + (MSE / sigma) * p_df['Yp']

    p_df[name] = p_df[name].apply(lambda x: 0 if x < 0 else x)
    p_df = p_df.drop('Yp', axis=1)
      
    return p_df.round(1)

In [ ]:
def save_as_excel_table(excel_file_name, sheet_name, df, float_num):
    """Save dataframe as xlsx file
        excel_file_name: tr,wihtout .xlsx
        
        sheet_name: any, of sheet in file
        
        df: dataframe, ta
        
        float_num: numbers of float nulls. In format - '00' - means two zeros"""
    
    from styleframe import StyleFrame, Styler, utils
    import openpyxl
    
    with StyleFrame.ExcelWriter('{}.xlsx'.format(excel_file_name)) as writer:
        
        sf=StyleFrame(df)
        
        # table
        sf.apply_column_style(cols_to_style=df.columns, \
                              styler_obj=Styler(bg_color=utils.colors.white, \
                                                bold=False, \
                                                font=utils.fonts.calibri,\
                                                font_size=11),
                              style_header=False)
        
        # header
        sf.apply_headers_style(styler_obj=Styler(bg_color=utils.colors.white, \
                                                 bold=True, \
                                                 font_size=11, \
                                                 font_color=utils.colors.black,\
                                                 number_format=utils.number_formats.general, \
                                                 protection=False))
        
        
        sf.to_excel(writer, sheet_name=str(sheet_name), index = False)
        writer.save()
        
    import openpyxl  
    wb = openpyxl.load_workbook(filename = '{}.xlsx'.format(excel_file_name))
    ws = wb['{}'.format(str(sheet_name))]
    
    
    for x in range(1,51):
        for y in range(1,51):
            ws.cell(row=x, column=y).number_format = '0.{}'.format(float_num)
    wb.save('{}.xlsx'.format(excel_file_name))
             
        
    return(print('result in file "{}.xlsx"'.format(excel_file_name)))